# LUME-services demo
In this notebook, we will configure LUME-services to use the service configuration used to launch our docker-compose services. Make sure you've completed all steps outlined in https://slaclab.github.io/lume-services/demo/.

In [1]:
import logging
logging.basicConfig(level=logging.INFO)  # Lets check the logs

## Configure services
LUME-services is packages with a configuration utility that reads environment variables and initializes services:

In [2]:
from lume_services import config
config.configure()

INFO:lume_services.config:Configuring LUME-services environment...
INFO:lume_services.config:Environment configured.


## if you're running this many time, creation will fail because of uniqueness... You can reset since this is a dev server

In [3]:
#model_db_service._reset()

## Create a model
The LUME-services Model provides an API to all model services and facilitates all model operations.

In [4]:
from lume_services.models import Model

model = Model.create_model(
    author = "Aman Singh Thakur",
    laboratory = "slac",
    facility = "lcls",
    beampath = "cu",
    description = "lcls-cu-inj-nn"
)
model

INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.author = :author_1 AND model.laboratory = :laboratory_1 AND model.facility = :facility_1 AND model.beampath = :beampath_1 AND model.description = :description_1
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.model_id = :model_id_1


Model(metadata=Model(                     model_id=8,                     created=datetime.datetime(2023, 6, 29, 19, 49, 12),                     author='Aman Singh Thakur'),                     laboratory='slac',                     facility='lcls',                     beampath='cu',                     description='lcls-cu-inj-nn'                 ), deployment=None, results=None)

## Create a project
Workflows are organized by the Prefect scheduler into different projects. Below, we access the configured services directly (TODO create project registry utility)

In [5]:
model_db_service = config.context.model_db_service()
scheduling_service = config.context.scheduling_service()

In [6]:
### NOTE: The below cell will raise an error if run 2x

In [7]:
# create a project
project_name = model_db_service.store_project(
    project_name="lcls-cu-inj-nn5", description="my_description"
)
scheduling_service.create_project("lcls-cu-inj-nn5")

INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO project (project_name, description) VALUES (:project_name, :description)


IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry 'lcls-cu-inj-nn5' for key 'project.PRIMARY'")
[SQL: INSERT INTO project (project_name, description) VALUES (%(project_name)s, %(description)s)]
[parameters: {'project_name': 'lcls-cu-inj-nn5', 'description': 'my_description'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

You can now find this project in you Prefect UI at http://localhost:8080


![project](https://slaclab.github.io/lume-services/files/project_nav.png)

## Create a deployment for your model
Replace `source_path` with the path to your release tarball below:

In [8]:
#source_path = "https://github.com/jacquelinegarrahan/lume-lcls-cu-inj-nn/releases/download/v0.0.12/lume_lcls_cu_inj_nn-0.0.12.tar.gz"

source_path = "https://github.com/singh96aman/lume-lcls-cu-inj-nn/releases/download/v0.0.14/lume_lcls_cu_inj_nn-0.0.14.tar.gz"

# populates local channel
model.store_deployment(source_path, project_name="lcls-cu-inj-nn5")
model

INFO:lume_services.models.model:installing package
INFO:lume_services.environment.solver:https://github.com/singh96aman/lume-lcls-cu-inj-nn/releases/download/v0.0.14/lume_lcls_cu_inj_nn-0.0.14.tar.gz saved to /tmp/tmp4iovld7u/lume_lcls_cu_inj_nn-0.0.14.tar.gz
INFO:lume_services.environment.solver:Version 0.0.14 of lume_lcls_cu_inj_nn already installed.
2023-06-29 14:49:06.309414: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT deployment.deployment_id, deployment.version, deployment.deploy_date, deployment.package_import_name, deployment.asset_dir, deployment.source, deployment.sha256, deployment.image, deployment.is_live, deployment.model_i

Flow URL: http://localhost:8080/test/flow/42330f3f-4103-4028-b697-90c47b4f12db
 └── ID: bd6d3c65-0e26-430e-a453-c81854c8bfbd
 └── Project: lcls-cu-inj-nn5
 └── Labels: ['lume-services']


Model(metadata=Model(                     model_id=8,                     created=datetime.datetime(2023, 6, 29, 19, 49, 12),                     author='Aman Singh Thakur'),                     laboratory='slac',                     facility='lcls',                     beampath='cu',                     description='lcls-cu-inj-nn'                 ), deployment=Deployment(metadata=Deployment(                 deployment_id=7,                 model_id=8,                 version='0.0.14',                 deploy_date=datetime.datetime(2023, 6, 29, 21, 49, 7)),                 asset_dir=None,                 source='https://github.com/singh96aman/lume-lcls-cu-inj-nn/releases/download/v0.0.14/lume_lcls_cu_inj_nn-0.0.14.tar.gz',                 sha256='8596f14feb55b6bfdcea128eedb08901857fdbe2c17b277b8da6af5f145af1e6',                 image='scr.svc.stanford.edu/aman96/lume-lcls-cu-inj-nn:v0.0.14',                 is_live=True                 package_import_name='lume_lcls_cu_inj_nn'         

## Run the Prefect workflow directly

In [9]:
flow_run = model.deployment.flow.prefect_flow.run(**{
                        "distgen:r_dist:sigma_xy:value": 0.4130, 
                        "distgen:total_charge:value": 250.0, 
                        "distgen:t_dist:length:value":7.499772441611215, 
                        "SOL1:solenoid_field_scale": 0.17, 
                        "CQ01:b1_gradient":-0.0074,
                        "SQ01:b1_gradient": -0.0074,
                        "L0A_phase:dtheta0_deg": -8.8997,
                        "L0A_scale:voltage": 70000000.0,
                        "distgen:t_dist:length:value": 7.499772441611215,
                        "end_mean_z": 4.6147002
    }
)


[2023-06-29 14:49:15-0700] INFO - prefect.FlowRunner | Beginning Flow run for 'lume-lcls-cu-inj-nn'


INFO:prefect.FlowRunner:Beginning Flow run for 'lume-lcls-cu-inj-nn'


[2023-06-29 14:49:15-0700] INFO - prefect.TaskRunner | Task 'distgen:r_dist:sigma_xy:value': Starting task run...


INFO:prefect.TaskRunner:Task 'distgen:r_dist:sigma_xy:value': Starting task run...


[2023-06-29 14:49:15-0700] INFO - prefect.TaskRunner | Task 'distgen:r_dist:sigma_xy:value': Finished task run for task with final state: 'Success'


...

INFO:prefect.TaskRunner:Task 'save_db_result': Starting task run...


[2023-06-29 14:49:17-0700] INFO - prefect.TaskRunner | Task 'save_db_result': Finished task run for task with final state: 'Success'


INFO:prefect.TaskRunner:Task 'save_db_result': Finished task run for task with final state: 'Success'


[2023-06-29 14:49:17-0700] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


INFO:prefect.FlowRunner:Flow run SUCCESS: all reference tasks succeeded


## Run the workflow inside the service cluster
We can use the model interface to directly deploy workflows. When sourcing our environment (`docs/examples/demo.env`), we defined a mount point for the file system using the alias `/lume-services/data`. Let's kick off this workflow and save the file output to that directory. 
After running the next cell, you'll be able to see the running container in your docker desktop and examine the flow using the Prefect UI at http://localhost:8080/default?flows.

In [10]:
model.run(
    parameters = {
                        "distgen:r_dist:sigma_xy:value": 0.4130, 
                        "distgen:total_charge:value": 250.0, 
                        "distgen:t_dist:length:value":7.499772441611215, 
                        "SOL1:solenoid_field_scale": 0.17, 
                        "CQ01:b1_gradient":-0.0074,
                        "SQ01:b1_gradient": -0.0074,
                        "L0A_phase:dtheta0_deg": -8.8997,
                        "L0A_scale:voltage": 70000000.0,
                        "distgen:t_dist:length:value": 7.499772441611215,
                        "end_mean_z": 4.6147002
    }
)

In [11]:
model.deployment.flow

Flow(name='lume-lcls-cu-inj-nn', flow_id='bd6d3c65-0e26-430e-a453-c81854c8bfbd', project_name='lcls-cu-inj-nn5', prefect_flow=<Flow: name="lume-lcls-cu-inj-nn">, parameters=None, mapped_parameters=None, task_slugs=None, labels=['lume-services'], image='scr.svc.stanford.edu/aman96/lume-lcls-cu-inj-nn:v0.0.14')

# Get results:

In [ ]:
results = model.get_results()
results

In [ ]:
results_df = model.get_results_df()
results_df

## Load model using model id
Once your model has been registered, you can use the `Model` api object to interact with your model without running the above registration steps. Let's load a new model object using the model_id we registered above.

In [ ]:
from importlib import reload 
import lume_services
reload(lume_services.models)
from lume_services.models import Model

#model_id = model.metadata.model_id
loaded_model = Model(model_id=1)

In [ ]:
loaded_model.metadata

## Load existing model object
Loading a model using the load_deployment method without passing a deployment_id will load the latest deployment registered for the model.

In [ ]:
loaded_model.load_deployment()
loaded_model.deployment

In [ ]:
result = loaded_model.run_and_return(
    parameters={
                        "distgen:r_dist:sigma_xy:value": 0.4130, 
                        "distgen:total_charge:value": 250.0, 
                        "distgen:t_dist:length:value":7.499772441611215, 
                        "SOL1:solenoid_field_scale": 0.17, 
                        "CQ01:b1_gradient":-0.0074,
                        "SQ01:b1_gradient": -0.0074,
                        "L0A_phase:dtheta0_deg": -8.8997,
                        "L0A_scale:voltage": 70000000.0,
                        "distgen:t_dist:length:value": 7.499772441611215,
                        "end_mean_z": 4.6147002
    },
    task_name="save_db_result" # Want to get the result from the save_db_result task
)
result

In [ ]:
results_df = loaded_model.get_results_df()
results_df